In [1]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB


In [3]:
# PARAMETROS

nPeriods = 5
nTypes = 3
maxStart0 = 5

## periodo de tempo
periodHours = [6, 3, 6, 3, 6]
## demanda
demand = [15000, 30000, 25000, 40000, 27000]


## quantidade de geradores
generators = [12, 10, 5]
## Saída minima
minLoad = [850, 1250, 1500]
## Saída maxima
maxLoad = [2000, 1750, 4000]

##custo por hora
perHourCost = [1000, 2600, 3000]
##custo por MWh
perMWhCost = [2, 1.3, 3]
##custo de ligar
startupCost = [2000, 1000, 500]

In [4]:
#VARIAVEIS

model = gp.Model('PowerGeneration')
ngen = model.addVars(nTypes, nPeriods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(nTypes, nPeriods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(nTypes, nPeriods, vtype=GRB.CONTINUOUS, name="genoutput")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-17


In [5]:
# RESTRIÇÕES

## Geradores disponiveis
numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(nTypes) for period in range(nPeriods))

## Limite dos geradores
minOutput = model.addConstrs((output[type, period] >= minLoad[type] * ngen[type, period])
                              for type in range(nTypes) for period in range(nPeriods))

maxOutput = model.addConstrs((output[type, period] <= maxLoad[type] * ngen[type, period])
                              for type in range(nTypes) for period in range(nPeriods))


## Demanda
meetDemand = model.addConstrs(gp.quicksum(output[type, period] for type in range(nTypes)) >= demand[period]
                               for period in range(nPeriods))

## Reserva
reserve = model.addConstrs(gp.quicksum(maxLoad[type]*ngen[type, period] for type in range(nTypes)) >= 1.15*demand[period]
                    for period in range(nPeriods))

## Inicio
startup0 = model.addConstrs((ngen[type,0] <= maxStart0 + nstart[type,0])
                            for type in range(nTypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(nTypes) for period in range(1,nPeriods))


In [6]:
# OBJETIVO

active = gp.quicksum(perHourCost[type]*periodHours[period]*ngen[type,period]
                    for type in range(nTypes) for period in range(nPeriods))

perMwh = gp.quicksum(perMWhCost[type]*periodHours[period]*(output[type,period] - minLoad[type]*ngen[type,period])
                       for type in range(nTypes) for period in range(nPeriods))

startupObj = gp.quicksum(startupCost[type]*nstart[type,period]
                         for type in range(nTypes) for period in range(nPeriods))

model.setObjective(active + perMwh + startupObj)

In [7]:
# Otimização do Gurobi
model.write('junk.lp')
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 70 rows, 45 columns and 147 nonzeros
Model fingerprint: 0xd010fb19
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]
Presolve removed 16 rows and 1 columns
Presolve time: 0.00s
Presolved: 54 rows, 44 columns, 130 nonzeros
Variable types: 0 continuous, 44 integer (0 binary)
Found heuristic solution: objective 1123913.3000
Found heuristic solution: objective 1084300.0000

Root relaxation: objective 9.995143e+05, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [8]:
#ANALISE

## Solução otima para o número de geradores que devem ficar ativos em cada periodo. 
rows = ["Tipo " + str(t) for t in range(nTypes)]
units = pd.DataFrame(columns=range(nPeriods), index=rows, data=0.0)

for t in range(nTypes):
    for p in range(nPeriods):
        units.loc["Tipo "+str(t), p] = ngen[t,p].x
units

,0,1,2,3,4
Tipo 0,12.0,12.0,12.0,12.0,12.0
Tipo 1,3.0,8.0,8.0,9.0,9.0
Tipo 2,0.0,0.0,0.0,2.0,0.0


In [9]:
## Solução otima para o número de geradores que devem ser ligados em cada periodo.
startups = pd.DataFrame(columns=range(nPeriods), index=rows, data=0.0)

for t in range(nTypes):
    for p in range(nPeriods):
        startups.loc["Tipo "+str(t), p] = int(nstart[t,p].x)
startups

,0,1,2,3,4
Tipo 0,7.0,0.0,0.0,0.0,0.0
Tipo 1,0.0,5.0,0.0,1.0,0.0
Tipo 2,0.0,0.0,0.0,2.0,0.0
